In [ ]:
import nltk
nltk.download('all')
from nltk.corpus import gutenberg
import pandas as pd
import numpy as np


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
with open('datasets/hamlet.txt', 'r') as fp:
    text = fp.read().lower()

tknzr = Tokenizer()
tknzr.fit_on_texts([text])
total_words = len(tknzr.word_index) + 1

In [ ]:
total_words

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tknzr.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
input_sequences.shape

In [ ]:
#Create predictors and label
import tensorflow as tf

x,y = input_sequences[:,:-1],input_sequences[:,-1]


In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
y_train.shape

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout(rate=0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), verbose=1)

In [ ]:
def predict_next_word(model, tknzr, text, max_sequence_len):
    token_list = tknzr.texts_to_sequences([text])[0]
    if len(token_list) > max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=1)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tknzr.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
ip_text = 'to be or not to be'
max_sequence_len = model.input_shape[1] + 1
predict_next_word(model, tknzr, ip_text, max_sequence_len)

In [ ]:
import pickle
model.save('datasets/next_word_lstm.h5')
with open('datasets/next_word_lstm_tknzr.pkl', 'wb') as fp:
    pickle.dump(tknzr, fp, pickle.HIGHEST_PROTOCOL)